In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import joblib
from true_model import true_model
from copy import deepcopy
import MNL_or_EC as tp
import LC_MNL as LM
import build_new_data as bnd
import assortment_process as at
import warnings
import torch
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from alpermodel_1 import conicmc
#from transformer_using_train_dataset_20220801 import trained_model
from collections import Counter
from  new_transformer_combination import Modifiled_Transformer

## Generating the new traing data or testing data

In [ ]:
choice_size = 30
order_num = 10000
file_name = "test"

#c_rate is \gamma
for c_rate in [5,10,20,50,100,200]:
    print(c_rate)
    next_train_data = bnd.generate_data(order_num,int(choice_size+c_rate))
    next_train_data.to_csv(file_name+"_dataset\\"+str(c_rate)+'_'+str(order_num)+'.csv',index=False)
    
    #transformer get lables for data
    pro = true_model.true_model(next_train_data,order_num)
    pro = pro.reshape(order_num,int(choice_size+c_rate))
    label = np.zeros(shape = pro.shape)
    for i in range(pro.shape[0]):
        index = np.argmax(np.random.multinomial(1,pro[i]/np.sum(pro[i])))
        label[i,index] = 1
    label = label[:,0:choice_size]
    pro = pro[:,0:choice_size]

    del_feature = ['orderid']
    features = [i for i in next_train_data.columns if i not in del_feature]
    data_numpy = np.array(next_train_data[features]).reshape(order_num,int(choice_size+c_rate),-1)
    np.save(file_name+"_dataset_numpy\\data_"+str(c_rate)+'_'+str(order_num)+'.npy',data_numpy[:,0:choice_size,:])
    np.save(file_name+"_dataset_numpy\\label_"+str(c_rate)+'_'+str(order_num)+'.npy',label)
    np.save(file_name+"_dataset_numpy\\pro_"+str(c_rate)+'_'+str(order_num)+'.npy',pro)
    print(pro[0,:])

    ##Transform into the numpy data
    del_feature = ['orderid']
    features = [i for i in next_train_data.columns if i not in del_feature]
    data_np = np.array(next_train_data[features])
    data_np_cate = np.zeros(shape = data_np.shape)
    for idx,item in enumerate([11, 7, 97, 63, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1]):
        if item !=1 :
            data_np_cate[:,idx] = data_np[:,idx]
            data_np[:,idx] = 1

    scaler = MinMaxScaler().fit(data_np)
    data_np = scaler.transform(data_np)
    for idx,item in enumerate([11, 7, 97, 63, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1]):
        if item != 1:
            data_np[:,idx] = 1

    data_np = data_np.reshape(-1,int(choice_size+c_rate),len(features))
    data_np_cate = data_np_cate.reshape(-1,int(choice_size+c_rate),len(features))
    data = np.array([data_np_cate[:,0:choice_size,:],data_np[:,0:choice_size,:]])

    #Incorporate the assortment information for DeemFM-a
    data_temp = np.zeros((order_num, choice_size, 9))
    data_cate_temp = np.zeros((order_num, choice_size, 2))
    for i in range(order_num):
        min1 = np.min(data[1,i,:,5])
        max1 = np.max(data[1,i,:,5])
        mean1 = np.mean(data[1,i,:,5])
        min2 = np.min(data[1,i,:,6])
        max2 = np.max(data[1,i,:,6])
        mean2 = np.mean(data[1,i,:,6])
        min3 = np.min(data[1,i,:,7])
        max3 = np.max(data[1,i,:,7])
        mean3 = np.mean(data[1,i,:,7])
        data_temp[i,:,0] = min1
        data_temp[i,:,1] = max1
        data_temp[i,:,2] = mean1
        data_temp[i,:,3] = min2
        data_temp[i,:,4] = max2
        data_temp[i,:,5] = mean2
        data_temp[i,:,6] = min3
        data_temp[i,:,7] = max3
        data_temp[i,:,8] = mean3 
        for j in range(choice_size):
            if data[1,i,j,6] == np.min(data[1,i,:,6]):
                data_cate_temp[i,j,0] = 1
            if data[1,i,j,6] == np.max(data[1,i,:,6]):
                data_cate_temp[i,j,1] = 1
    data_with_assort_info = np.array([np.concatenate([data[0],np.zeros((order_num, choice_size, 9)),data_cate_temp],axis = 2),np.concatenate([data[1],data_temp,np.ones((order_num,choice_size,2))],axis = 2)])

    np.save(file_name+'_dataset_numpy_normalised\\data_'+str(c_rate)+'_'+str(order_num)+'.npy',data)
    np.save(file_name+'_dataset_numpy_normalised\\label_'+str(c_rate)+'_'+str(order_num)+'.npy',label)
    np.save(file_name+'_dataset_numpy_normalised\\data_with_assort_info_'+str(c_rate)+'_'+str(order_num )+'.npy',data_with_assort_info)

## Generating the decision-test data set

In [4]:
for c_rate in [200]:
    print(c_rate)
    next_valid_data = bnd.generate_data(1000,100+c_rate)
    del_feature = ['orderid','alternative','orderlabel']
    features = [i for i in next_valid_data.columns if i not in del_feature]
    data_np = np.array(next_valid_data[features])
    scaler = MinMaxScaler().fit(data_np)
    data_np_cate = np.zeros(shape = data_np.shape)
    for idx,item in enumerate([11, 7, 97, 63, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1]):
        if item !=1 :
            data_np_cate[:,idx] = data_np[:,idx]
            data_np[:,idx] = 1
    data_np = scaler.transform(data_np)
    for idx,item in enumerate([11, 7, 97, 63, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1]):
        if item != 1:
            data_np[:,idx] = 1

    data_np = data_np.reshape(-1,100+c_rate,len(features))
    data_np_cate = data_np_cate.reshape(-1,100+c_rate,len(features))
    valid_data = np.array([data_np_cate,data_np])
    next_valid_data.to_csv('new_products\\next_valid_data_'+str(c_rate)+'.csv',index=False)
    np.save('new_products\\valid_data_'+str(c_rate)+'.npy',valid_data)

200


## 1 MNL-Based Choice Model

### 1.1 Training the MNL model

In [ ]:
PATH = "trained_models\\MNL_parameters_{}_{}_{}.pt".format(c_rate,order_num,len(feature_column))
model_MNL = tp.train_data(data,label,NUM_EPOCHS = 100,BATCH_SIZE = 32,LR = 0.01,path = PATH)

### 1.2 Prediction Results

In [4]:
PATH = "trained_models\\MNL_parameters_{}_{}_{}.pt".format(c_rate,order_num,len(feature_column))
model_MNL = torch.load(PATH)

X = torch.from_numpy(test_data[0][:,:,feature_column]).to(torch.long)
weight = torch.from_numpy(test_data[1][:,:,feature_column]).to(torch.float)
utility_MNL,u0 = model_MNL([X,weight])
utility_MNL = utility_MNL.detach().numpy()
u0 = float(u0[0])
pro_MNL = np.zeros(shape = utility_MNL.shape)
for i in range(pro_MNL.shape[0]):
    pro_MNL[i] = np.exp(utility_MNL[i])/(np.sum(np.exp(utility_MNL[i]))+np.exp(u0))

ratio = 0
high_low_ratio = []
for i in range(pro_MNL.shape[0]):
    ratio = ratio + (np.max(pro_MNL[i])- np.min(pro_MNL[i]))/np.mean(pro_MNL[i])
    #index_array = np.argsort(-test_price[i])
    #high_low_ratio.append(pro_MNL[i,index_array[1]]/pro_MNL[i,index_array[0]])

rmse = []
count_number1 = 0
count_number3 = 0
for i in range(pro_MNL.shape[0]):
    rmse.append(np.sum((pro_MNL[i] - pro[i,0:30])**2) + (np.sum(pro_MNL[i]) - np.sum(pro[i,0:30]))**2)
    pro_index = np.argsort(-pro_MNL[i])
    if np.sum(test_label[i,:]) > 0:
        if np.argmax(test_label[i,:]) in list(pro_index[0:1]):
            count_number1 += 1
        if np.argmax(test_label[i,:]) in list(pro_index[0:3]):
            count_number3 += 1
print(" MNL Top1-acc: {:.4}".format(count_number1/np.sum(test_label)))
print(" MNL Top3-acc: {:.4}".format(count_number3/np.sum(test_label)))
print(" Averaged raito of MNL: {}".format(ratio/10000))
print("RMSE of MNL: {:.4}".format(np.sqrt(sum(rmse)/(10000*31))))

 MNL Top1-acc: 0.1348
 MNL Top3-acc: 0.309
 Averaged raito of MNL: 4.79226342945531
RMSE of MNL: 0.03939


### 1.3 Assotments Results

In [7]:
PATH = "trained_models\\MNL_parameters_{}_{}_{}.pt".format(c_rate,order_num,len(feature_column))
model_MNL = torch.load(PATH)

X = torch.from_numpy(valid_data[0]).to(torch.long)
weight = torch.from_numpy(valid_data[1]).to(torch.float)
utility_MNL,u0 = model_MNL([X,weight])
utility_MNL = utility_MNL.detach().numpy()
u0 = float(u0[0])

for cadinality in [30,50]:
    print("{}:".format(cadinality))
    warnings.filterwarnings('ignore')
    assortment_MNL = []
    for i in tqdm(range(utility_MNL.shape[0])):
        assortment_MNL.append(at.MNL(np.exp(utility_MNL[i,0:100]-u0),price[i,0:100],capacity = cadinality))
    products = []
    price_MNL = []
    for idx,item in enumerate(assortment_MNL):
        for j in range(utility_MNL.shape[1]):
            if j in item:
                products.append(idx*utility_MNL.shape[1]+j)
                price_MNL.append(price[idx,j])
            elif j >= 100:
                products.append(idx*utility_MNL.shape[1]+j)
                price_MNL.append(0)
    valid_pro = true_model.true_model(next_valid_data.loc[products],1000)
    profit = np.dot(valid_pro,np.array(price_MNL))/1000
    print("profit: {}".format(profit))
    assortment_array = np.zeros((1000,100))
    for i,item in enumerate(assortment_MNL):
        for j in item:
            assortment_array[i,j] = 1
    np.save("Assortment\\MNL_assortment_{}_{}_{}_{}.npy".format(c_rate,order_num,len(feature_column),cadinality),assortment_array)
    a = np.sum(assortment_array,axis = 1)
    print(np.min(a),np.mean(a),np.max(a))

30:


100%|██████████| 1000/1000 [14:52<00:00,  1.12it/s]


profit: 77.52122141846797
12.0 29.937 30.0
50:


100%|██████████| 1000/1000 [23:22<00:00,  1.40s/it]


profit: 72.64559436395216
12.0 49.663 50.0


## 2 DeepFM-based Choice Model

### 2.1 Training the DeepFM model

In [5]:
#Separate data
total = data.shape[1]
indices = np.random.permutation(total)
k_fold = 5
training_data = data[:,indices[0:int(total*(k_fold-1)/k_fold)],:,:]
training_label = label[indices[0:int(total*(k_fold-1)/k_fold)],:]
training_valid_data = data[:,indices[int(total*(k_fold-1)/k_fold):],:,:]
training_valid_label = label[indices[int(total*(k_fold-1)/k_fold):],:]
weight = int((training_label.shape[0]*training_label.shape[1]-np.sum(training_label))/np.sum(training_label))

In [ ]:
#train the model
PATH = "trained_models\\DeepFM_parameters_{}_{}_{}.pt".format(c_rate,order_num,len(feature_column))
model_DeepFM = tp.train_DeepFM(training_data,training_label,training_valid_data,training_valid_label,weight1 = weight,path = PATH,LR = 0.05)

### 2.2 Prediction Results

In [7]:
PATH = "trained_models\\DeepFM_parameters_{}_{}_{}.pt".format(c_rate,order_num,len(feature_column))
model_DeepFM = torch.load(PATH)
purchase_pro = np.sum(label)/label.shape[0]

X = torch.from_numpy(test_data[0][:,:,feature_column]).to(torch.long)
weight = torch.from_numpy(test_data[1][:,:,feature_column]).to(torch.float)
utility_DeepFM = model_DeepFM([X,weight])
pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()

ratio = 0
high_low_ratio = []
for i in range(pro_DeepFM.shape[0]):
    ratio = ratio + (np.max(pro_DeepFM[i])- np.min(pro_DeepFM[i]))/np.mean(pro_DeepFM[i])
    # index_array = np.argsort(-valid_price[i])
    # high_low_ratio.append(pro_DeepFM[i,index_array[1]]/pro_DeepFM[i,index_array[0]])

rmse = []
count_number1 = 0
count_number3 = 0
for i in range(pro_DeepFM.shape[0]):
    temp_pro = purchase_pro*pro_DeepFM[i]/np.sum(pro_DeepFM[i])
    rmse.append(np.sum((temp_pro - pro[i,0:30])**2) + (np.sum(temp_pro) - np.sum(pro[i,0:30]))**2)
    pro_index = np.argsort(-pro_DeepFM[i])
    if np.sum(test_label[i,:]) > 0:
        if np.argmax(test_label[i,:]) in list(pro_index[0:1]):
            count_number1 += 1
        if np.argmax(test_label[i,:]) in list(pro_index[0:3]):
            count_number3 += 1
print(" DeepFM Top1-acc: {:.4}".format(count_number1/np.sum(test_label)))
print(" DeepFM Top3-acc: {:.4}".format(count_number3/np.sum(test_label)))
print(" Averaged raito of DeepFM: {}".format(ratio/10000))
print("RMSE of DeepFM: {:.4}".format(np.sqrt(sum(rmse)/(10000*31))))

 DeepFM Top1-acc: 0.2566
 DeepFM Top3-acc: 0.5236
 Averaged raito of DeepFM: 3.6644734484434127
RMSE of DeepFM: 0.02973


### 2.3 Assortment Results

In [3]:
PATH = "trained_models\\DeepFM_parameters_{}_{}_{}.pt".format(c_rate,order_num,len(feature_column))
model_DeepFM = torch.load(PATH)

X = torch.from_numpy(valid_data[0]).to(torch.long)
weight = torch.from_numpy(valid_data[1]).to(torch.float)
utility_DeepFM  = model_DeepFM([X,weight])
pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()

for cadinality in [10,30,50]:
    warnings.filterwarnings('ignore')
    assortment_DeepFM = []
    for i in range(pro_DeepFM.shape[0]):
        #length = len(assortment_ml[i])
        revenue = pro_DeepFM[i,0:100]*price[i,0:100]
        sort_index = np.argsort(-revenue)
        assortment_DeepFM.append(sort_index[0:cadinality])

    products = []
    price_DeepFM = []
    for idx,item in enumerate(assortment_DeepFM):
        for j in range(valid_data.shape[2]):
            if j in item:
                products.append(idx*valid_data.shape[2]+j)
                price_DeepFM.append(price[idx,j])
            elif j >= 100:
                products.append(idx*valid_data.shape[2]+j)
                price_DeepFM.append(0)
    valid_pro = true_model.true_model(next_valid_data.loc[products],1000)
    average_profit_ml = np.dot(valid_pro,np.array(price_DeepFM))/1000
    print(cadinality,average_profit_ml)

10 150.451583129403
30 120.29881388711698
50 92.92911028833726


## 3 DeepFM-a based choice model

### 3.1 Training the DeepFM-a model

In [8]:
#load data
total = data_with_assort_info.shape[1]
indices = np.random.permutation(total) 
k_fold = 5
training_data = data_with_assort_info[:,indices[0:int(total*(k_fold-1)/k_fold)],:,:]
training_label = label[indices[0:int(total*(k_fold-1)/k_fold)],:]
training_valid_data = data_with_assort_info[:,indices[int(total*(k_fold-1)/k_fold):],:,:]
training_valid_label = label[indices[int(total*(k_fold-1)/k_fold):],:]
weight = int((training_label.shape[0]*training_label.shape[1]-np.sum(training_label))/np.sum(training_label))

In [ ]:
#train model
PATH = "trained_models\\DeepFM_parameters_with_assortment_{}_{}_{}.pt".format(c_rate,order_num,len(feature_column))
model_DeepFM = tp.train_DeepFM(training_data,training_label,training_valid_data,training_valid_label,model_name = 'Assortment',weight1 = weight,LR = 0.05,path = PATH,feature_column = feature_column_swapping)

### 3.2 Prediction Results

In [10]:
PATH = "trained_models\\DeepFM_parameters_with_assortment_{}_{}_{}.pt".format(c_rate,order_num,len(feature_column))
model_DeepFM = torch.load(PATH)
purchase_pro = np.sum(label)/label.shape[0]

X = torch.from_numpy(test_data_with_assort_info[0][:,:,feature_column_swapping]).to(torch.long)
weight = torch.from_numpy(test_data_with_assort_info[1][:,:,feature_column_swapping]).to(torch.float)
utility_DeepFM  = model_DeepFM([X,weight])
pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()

ratio = 0
high_low_ratio = []
for i in range(pro_DeepFM.shape[0]):
    ratio = ratio + (np.max(pro_DeepFM[i])- np.min(pro_DeepFM[i]))/np.mean(pro_DeepFM[i])
    # index_array = np.argsort(-valid_price[i])
    # high_low_ratio.append(pro_DeepFM[i,index_array[1]]/pro_DeepFM[i,index_array[0]])

rmse = []
count_number1 = 0
count_number3 = 0
for i in range(pro_DeepFM.shape[0]):
    temp_pro = purchase_pro*pro_DeepFM[i]/np.sum(pro_DeepFM[i])
    rmse.append(np.sum((temp_pro - pro[i,0:30])**2) + (np.sum(temp_pro) - np.sum(pro[i,0:30]))**2)
    pro_index = np.argsort(-pro_DeepFM[i])
    if np.sum(test_label[i,:]) > 0:
        if np.argmax(test_label[i,:]) in list(pro_index[0:1]):
            count_number1 += 1
        if np.argmax(test_label[i,:]) in list(pro_index[0:3]):
            count_number3 += 1
print(" DeepFM-assort Top1-acc: {:.4}".format(count_number1/np.sum(test_label)))
print(" DeepFM-assort Top3-acc: {:.4}".format(count_number3/np.sum(test_label)))
print(" Averaged raito of DeepFM-assort: {}".format(ratio/10000))
print("RMSE of DeepFM-assort: {:.4}".format(np.sqrt(sum(rmse)/(10000*31))))

 DeepFM-assort Top1-acc: 0.2587
 DeepFM-assort Top3-acc: 0.5252
 Averaged raito of DeepFM-assort: 3.6495547036886213
RMSE of DeepFM-assort: 0.02974


### 3.3 Assortment Results

In [9]:
for cadinality in [10,30,50]:
    PATH = "trained_models\\DeepFM_parameters_with_assortment_{}_{}_{}.pt".format(c_rate,order_num,len(feature_column))
    model_DeepFM = torch.load(PATH)
    
    model_DeepFM1 = torch.load("trained_models\\DeepFM_parameters_{}_{}_{}.pt".format(c_rate,order_num,len(feature_column)))
    X = torch.from_numpy(valid_data[0]).to(torch.long)
    weight = torch.from_numpy(valid_data[1]).to(torch.float)
    utility_DeepFM  = model_DeepFM1([X,weight])
    pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()
    assortment_DeepFM = []
    for i in range(pro_DeepFM.shape[0]):
        #length = len(assortment_ml[i])
        revenue = pro_DeepFM[i,0:100]*price[i,0:100]
        sort_index = np.argsort(-revenue)
        assortment_DeepFM.append(sort_index[0:cadinality])
    warnings.filterwarnings('ignore')
    assortment_ml = []
    for i in tqdm(range(1000)):
        assortment_ml.append(at.ml_assortment_swap(model_DeepFM,valid_data[:,i,0:100,:],price[i,0:100],assortment_DeepFM[i],capacity = cadinality,feature_column = feature_column_swapping))
    products = []
    price_ml = []
    for idx,item in enumerate(assortment_ml):
        for j in range(valid_data.shape[2]):
            if j in item:
                products.append(idx*valid_data.shape[2]+j)
                price_ml.append(price[idx,j])
            elif j >= 100:
                products.append(idx*valid_data.shape[2]+j)
                price_ml.append(0)
    valid_pro = true_model.true_model(next_valid_data.loc[products],1000)
    average_profit_ml = np.dot(valid_pro,np.array(price_ml))/1000
    print(cadinality,average_profit_ml)
    assortment_array = np.zeros((1000,100))
    for i,item in enumerate(assortment_ml):
        for j in item:
            assortment_array[i,j] = 1   
    np.save("Assortment\\DeepFM_assortment_swapping_{}_{}_{}_{}.npy".format(c_rate,order_num,len(feature_column),cadinality),assortment_array)

100%|██████████| 1000/1000 [22:36<00:00,  1.36s/it]


10 141.82146150912527


100%|██████████| 1000/1000 [1:23:51<00:00,  5.03s/it]   


30 118.30328563586444


100%|██████████| 1000/1000 [46:49<00:00,  2.81s/it] 


50 92.61878254975801


## 4 Transformer

### 4.1 Train the transformer model

In [ ]:
c_rate = 5
order_num = 20000
train_data = pd.read_csv('train_dataset_for_transformer\\{}_{}.csv'.format(c_rate,order_num))
weight_index = np.sum(train_data['label'])/(len(list(train_data['orderid'].unique()))-np.sum(train_data['label']))
if weight_index > 1:
    pt_weight = [1,int(weight_index)]
else:
    pt_weight = [int(1/weight_index),1]
pt_weight = [8,1]

In [ ]:
Modifiled_Transformer.train_model(train_data,pt_weight = pt_weight,model_name = "{}_{}".format(c_rate,order_num))

### 4.2 Obtaining the prediction accuracy

In [ ]:
test_data = pd.read_csv('test_dataset_for_transformer\\{}_{}.csv'.format(c_rate,10000))
label = np.load('test_dataset_numpy\\label_{}_{}.npy'.format(c_rate,10000))

In [ ]:
pro_pred = Modifiled_Transformer.predict(test_data[test_data.columns[0:18]],pt_weight = pt_weight,model_name = "{}_{}".format(c_rate,order_num))
pro_pred = pro_pred.detach().numpy()
total = 0
count = 0
for i in range(pro_pred.shape[0]):
    if np.sum(label[i,:]) == 1:
        total = total + 1
        if np.argmax(pro_pred[i,:]) == np.argmax(label[i,:]):
            count = count + 1
print(count/total)

### 4.3 Get the assortment

In [ ]:
model_DeepFM1 = torch.load('trained_models\\DeepFM_parameters_'+str(c_rate)+'_'+str(order_num )+'.pt')
X = torch.from_numpy(valid_data[0]).to(torch.long)
weight = torch.from_numpy(valid_data[1]).to(torch.float)
utility_DeepFM  = model_DeepFM1([X,weight])
pro_DeepFM = torch.sigmoid(utility_DeepFM).detach().numpy()
assortment_DeepFM = []
for i in range(pro_DeepFM.shape[0]):
    #length = len(assortment_ml[i])
    revenue = pro_DeepFM[i,0:100]*price[i,0:100]
    sort_index = np.argsort(-revenue)
    assortment_DeepFM.append(sort_index[0:30])
warnings.filterwarnings('ignore')

#assortment_transformer = []
#tasks = []
assortment_transformer = np.load("Assortment//Transformer_{}_{}.npy".format(c_rate,order_num),allow_pickle=True)

for i in tqdm(range(1000)):
    if len(assortment_transformer[i]) != 30:
        temp_df = next_valid_data[next_valid_data.orderid == i]
        temp_df.reset_index(drop = True,inplace = True)
        temp_df = temp_df.loc[0:99,:]
        temp_price = np.array(temp_df.loc[0:99,'totalPrice'])
        assortment_transformer[i] = at.transformer_assortment_swap("{}_{}".format(c_rate,order_num),temp_df,list(assortment_DeepFM[i]),pt_weight,temp_price)